In [1]:
import tensorflow as tf
import pathlib
from Utilities.directories import *

In [2]:
# Define directories to load raw data from and to save serialized data to
root_dir = pathlib.Path(data + "/example_extrect_ma")
save_dir = pathlib.Path(data + "/example_extract-tfecord")
dataset = tf.data.Dataset.list_files(str(root_dir/'*'), seed=10000)

In [8]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [9]:
print(_bytes_feature(b'some string'))

bytes_list {
  value: "some string"
}



In [10]:
def serialize_example(feature0, feature1):
    """
    Creates a tf.train.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.train.Example-compatible
    # data type.

    feature = {
        "feature0": _bytes_feature(feature0),
        "feature1": _int64_feature(feature1)
    }

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [11]:
# Test the serialization

serialized_example = serialize_example(feature0=b'test', feature1=15)
serialized_example

b'\n)\n\x14\n\x08feature0\x12\x08\n\x06\n\x04test\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x0f'

In [1]:
# Convert back

import tensorflow as tf
from Utilities.directories import lexis_abstract
import pathlib

features = {
    'abstract': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}
train_files = tf.data.Dataset.list_files(str(lexis_abstract) + "\*train*.tfrec")
print(list(train_files.as_numpy_iterator()))
test_file = str(pathlib.Path.joinpath(lexis_abstract, "validate-1-15276.tfrec"))
dataset = tf.data.TFRecordDataset(filenames=[test_file], compression_type="ZLIB")
raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())
print(parsed.features.feature['label'])

[b'E:\\MLData\\thesis\\Datasets\\1.Abstract-SingleClass\\train-1-122203.tfrec']
int64_list {
  value: 3
}



In [ ]:
import pathlib
from to_tfrecord import TFRecordsConverter
from Utilities.directories import data

# Convert to tfRecord
if __name__ == '__main__':
    output_dir = pathlib.Path.joinpath(data, "1.Abstract-SingleClass")
    converter = TFRecordsConverter(feature_stats, output_dir, 0.1, 0.1)
    converter.convert()

In [ ]:
# Load Huggingface transformer
from transformers import BertConfig, BertTokenizerFast

# Then what you need from tensorflow.keras
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from Utilities.directories import lexis_abstract
from tqdm import tqdm

train_files = tf.data.Dataset.list_files(file_pattern=str(lexis_abstract) + "\*train*.tfrec")
test_files = tf.data.Dataset.list_files(file_pattern=str(lexis_abstract) + "\*test*.tfrec")
validation_files = tf.data.Dataset.list_files(file_pattern=str(lexis_abstract) + "\*validate*.tfrec")

features = {
    'abstract': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}

test_dataset = tf.data.TFRecordDataset(filenames=test_files, compression_type="ZLIB")


def _parse_function(example_proto, features):
    return tf.io.parse_single_example(example_proto, features)

def select_data_from_record(record):
    x = record['abstract']
    y = record['label']
    return (x, y)

parsed_dataset = test_dataset.map(lambda record: _parse_function(record, features))
parsed_dataset = parsed_dataset.map(select_data_from_record)

for p in parsed_dataset:
    print(p)
    break


# https://colab.research.google.com/drive/1yWaLpCWImXZE2fPV0ZYDdWWI8f52__9A#scrollTo=BmMlC1i0COBW


